## <center> Word2Vec from Scratch using Tensorflow

In [0]:
from IPython.display import Image

Word Embeddings are dense vector representations of words in low dimensional vector space. Word2Vec is the most popular word embedding model.  The use of Word2Vec is to group words that semantically similar in vector space. It computes similarities mathematically. 

In [6]:
Image(url= "1.png")

There are two variants :
1. CBOW (Continuous Bag of Words) : It tries to predict a word on based of its neighbours.
2. SkipGram : It tries to predict the neighbours of a given word.

In [7]:
Image('Cbowskip.png')

# Steps:

1. Build a 3 layer neural network. 
2. The objective of network is to predict the neighbouring word given a word.
3. Remove the last layer and keep the input and hidden layer.
4. Now, input a word from within the vocabulary. The output given at the hidden layer is the ‘word embedding’ of the input word.

In [8]:
Image('4.png')

In [9]:
import numpy as np
import tensorflow as tf

In [0]:
corpus_raw = """Shah Rukh Khan (born 2 November 1965), also known by the initialism SRK, is an Indian actor, film producer, and television personality. Referred to in the media as the "Badshah of Bollywood", "King of Bollywood" and "King Khan", he has appeared in more than 80 Bollywood films, and earned numerous accolades, including 14 Filmfare Awards. The Government of India has awarded him the Padma Shri, the fourth-highest Indian civilian honour, and the Government of France the Officier of the Ordre des Arts et des Lettres, the second-degree of the honour, and the Chevalier of the Legion of Honour, the fifth degree of the honour. Khan has a significant following in Asia and the Indian diaspora worldwide. In terms of audience size and income, he has been described as one of the most successful film stars in the world.[a] Khan began his career with appearances in several television series in the late 1980s. He made his Bollywood debut in 1992 with Deewana. Early in his career, Khan was recognised for portraying villainous roles in the films Darr (1993), Baazigar (1993) and Anjaam (1994). He then rose to prominence after starring in a series of romantic films, including Dilwale Dulhania Le Jayenge (1995), Dil To Pagal Hai (1997), Kuch Kuch Hota Hai (1998), Mohabbatein (2000) and Kabhi Khushi Kabhie Gham... (2001). Khan went on to earn critical acclaim for his portrayal of an alcoholic in Devdas (2002), a NASA scientist in Swades (2004), a hockey coach in Chak De! India (2007) and a man with Asperger syndrome in My Name Is Khan (2010). His highest-grossing films include the comedies Chennai Express (2013), Happy New Year (2014), Dilwale (2015), and the crime film Raees (2017). Many of his films display themes of Indian national identity and connections with diaspora communities, or gender, racial, social and religious differences and grievances. As of 2015, Khan is co-chairman of the motion picture production company Red Chillies Entertainment and its subsidiaries, and is the co-owner of the Indian Premier League cricket team Kolkata Knight Riders and the Caribbean Premier League team Trinbago Knight Riders. He is a frequent television presenter and stage show performer. The media often label him as "Brand SRK" because of his many endorsement and entrepreneurship ventures. Khan's philanthropic endeavours have provided health care and disaster relief, and he was honoured with UNESCO's Pyramide con Marni award in 2011 for his support of children's education and the World Economic Forum's Crystal Award in 2018 for his leadership in championing women's and children's rights in India. He regularly features in listings of the most influential people in Indian culture, and in 2008, Newsweek named him one of their fifty most powerful people in the world.[7]"""
corpus_raw = corpus_raw.lower()

In [0]:
import re
keeplist1 = '[^a-zA-Z /.]'
keeplist2 = '[^a-zA-Z ]'
corpus_raw = re.sub(keeplist1,'',corpus_raw)
corpus_raw1 = re.sub(keeplist2,'',corpus_raw)


In [61]:
print(corpus_raw)

shah rukh khan born  november  also known by the initialism srk is an indian actor film producer and television personality. referred to in the media as the badshah of bollywood king of bollywood and king khan he has appeared in more than  bollywood films and earned numerous accolades including  filmfare awards. the government of india has awarded him the padma shri the fourthhighest indian civilian honour and the government of france the officier of the ordre des arts et des lettres the seconddegree of the honour and the chevalier of the legion of honour the fifth degree of the honour. khan has a significant following in asia and the indian diaspora worldwide. in terms of audience size and income he has been described as one of the most successful film stars in the world.a khan began his career with appearances in several television series in the late s. he made his bollywood debut in  with deewana. early in his career khan was recognised for portraying villainous roles in the films d

In [0]:
words = []
for word in corpus_raw1.split():
    if word != '.': 
        words.append(word)

In [0]:
words = set(words) 

In [0]:
words

In [0]:
word2int = {}
int2word = {}
vocab_size = len(words)

for i,word in enumerate(words):
    word2int[word] = i
    int2word[i] = word

In [66]:
print(word2int['personality'])

132


In [67]:
print(int2word[2])

ordre


In [0]:
raw_sentences = corpus_raw.split('.')
sentences = []
for sentence in raw_sentences:
    sentences.append(sentence.split())
                 

In [0]:
sentences

In [40]:
print(sentences)

[['shah', 'rukh', 'khan', 'born', 'november', 'also', 'known', 'by', 'the', 'initialism', 'srk', 'is', 'an', 'indian', 'actor', 'film', 'producer', 'and', 'television', 'personality'], ['referred', 'to', 'in', 'the', 'media', 'as', 'the', 'badshah', 'of', 'bollywood', 'king', 'of', 'bollywood', 'and', 'king', 'khan', 'he', 'has', 'appeared', 'in', 'more', 'than', 'bollywood', 'films', 'and', 'earned', 'numerous', 'accolades', 'including', 'filmfare', 'awards'], ['the', 'government', 'of', 'india', 'has', 'awarded', 'him', 'the', 'padma', 'shri', 'the', 'fourthhighest', 'indian', 'civilian', 'honour', 'and', 'the', 'government', 'of', 'france', 'the', 'officier', 'of', 'the', 'ordre', 'des', 'arts', 'et', 'des', 'lettres', 'the', 'seconddegree', 'of', 'the', 'honour', 'and', 'the', 'chevalier', 'of', 'the', 'legion', 'of', 'honour', 'the', 'fifth', 'degree', 'of', 'the', 'honour'], ['khan', 'has', 'a', 'significant', 'following', 'in', 'asia', 'and', 'the', 'indian', 'diaspora', 'worldw

In [0]:
data = []
WINDOW_SIZE = 2

def to_one_hot(data_point_index, vocab_size):
    temp = np.zeros(vocab_size)
    temp[data_point_index] = 1
    return temp


In [0]:
for sentence in sentences:
    for word_index, word in enumerate(sentence):
        for nb_word in sentence[max(word_index - WINDOW_SIZE, 0) : min(word_index + WINDOW_SIZE, len(sentence)) + 1] : 
            if nb_word != word:
                data.append([word, nb_word])

In [0]:
x_train = [] 
y_train = [] 
for data_word in data:
    x_train.append(to_one_hot(word2int[ data_word[0] ], vocab_size))
    y_train.append(to_one_hot(word2int[ data_word[1] ], vocab_size))


In [0]:
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

In [95]:
print(x_train.shape, y_train.shape)

(1600, 234) (1600, 234)


In [96]:
Image('3.jpeg')

In [0]:
x = tf.placeholder(tf.float32, shape=(None, vocab_size))
y_label = tf.placeholder(tf.float32, shape=(None, vocab_size))

In [0]:
EMBEDDING_DIM = 50 
W1 = tf.Variable(tf.random_normal([vocab_size, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([EMBEDDING_DIM]))
hidden_representation = tf.add(tf.matmul(x,W1), b1)

In [0]:
W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, vocab_size]))
b2 = tf.Variable(tf.random_normal([vocab_size]))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_representation, W2), b2))

In [100]:

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init) 

# define the loss function:
cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), reduction_indices=[1]))
# define the training step:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy_loss)
n_iters = 10000
# train for n_iter iterations
for _ in range(n_iters):
    sess.run(train_step, feed_dict={x: x_train, y_label: y_train})
    print('loss is : ', sess.run(cross_entropy_loss, feed_dict={x: x_train, y_label: y_train}))

loss is :  25.786749
loss is :  24.48413
loss is :  23.465803
loss is :  22.629145
loss is :  21.942188
loss is :  21.39728
loss is :  20.969189
loss is :  20.614784
loss is :  20.308199
loss is :  20.036951
loss is :  19.7939
loss is :  19.574028
loss is :  19.373302
loss is :  19.188374
loss is :  19.01655
loss is :  18.855738
loss is :  18.704336
loss is :  18.561115
loss is :  18.42511
loss is :  18.295557
loss is :  18.17185
loss is :  18.053497
loss is :  17.94011
loss is :  17.83134
loss is :  17.726927
loss is :  17.626623
loss is :  17.530212
loss is :  17.4375
loss is :  17.348272
loss is :  17.262321
loss is :  17.179422
loss is :  17.099354
loss is :  17.021893
loss is :  16.946844
loss is :  16.874022
loss is :  16.80327
loss is :  16.734446
loss is :  16.667442
loss is :  16.602139
loss is :  16.53845
loss is :  16.476292
loss is :  16.41559
loss is :  16.35627
loss is :  16.298271
loss is :  16.241539
loss is :  16.186012
loss is :  16.131645
loss is :  16.07839
loss is 

In [101]:
print(sess.run(W1))
print(sess.run(b1))

[[-0.6056982   0.08956975  0.05534233 ... -0.65028715 -0.30515024
  -2.3543074 ]
 [ 1.1817173  -1.004612    1.6556126  ...  1.3192261   0.26437244
  -0.23100509]
 [ 0.73276913 -0.5756581   0.79889375 ... -0.38353285  0.32838303
   0.7378773 ]
 ...
 [ 1.7676737  -0.16956493 -0.49572456 ... -0.87243533 -0.42700344
   1.6257907 ]
 [-0.59006774  1.2016637  -1.1572958  ...  1.0638204   0.05437628
  -0.32749566]
 [ 1.8121867  -0.4917963  -0.31050307 ... -1.1830015  -1.1144334
  -0.07019254]]
[ 0.47614297  0.733422    1.1873777   0.5225002   0.11652108 -0.7876135
  0.63917744 -0.18790215 -0.5549963   0.814935   -0.21167202 -1.5389814
 -0.1328337  -0.24606489 -0.64452857 -0.10201382  0.44401744  0.93171585
 -1.0507128   0.0722175   0.45767972  0.01592535  0.15432894 -0.6784056
  0.22591406  1.3447021   0.36593357 -0.33427316 -0.46711686  0.0959281
  0.51382595  0.26696995  0.26587895  0.62819445 -0.36916825  0.08469468
  0.15500312  0.26199296  0.02007828  0.9288328   0.0233091  -0.627338
  0.

In [0]:
vectors = sess.run(W1 + b1)

In [103]:
vectors

array([[-0.12955526,  0.8229917 ,  1.24272   , ..., -0.49400166,
        -0.17438617, -3.1906772 ],
       [ 1.6578603 , -0.27119   ,  2.8429904 , ...,  1.4755117 ,
         0.3951365 , -1.067375  ],
       [ 1.2089121 ,  0.1577639 ,  1.9862714 , ..., -0.22724736,
         0.4591471 , -0.09849256],
       ...,
       [ 2.2438166 ,  0.5638571 ,  0.69165313, ..., -0.7161498 ,
        -0.29623938,  0.78942084],
       [-0.11392477,  1.9350858 ,  0.03008187, ...,  1.2201059 ,
         0.18514034, -1.1638656 ],
       [ 2.2883296 ,  0.24162567,  0.8768746 , ..., -1.026716  ,
        -0.98366934, -0.9065624 ]], dtype=float32)

In [104]:
print(vectors[ word2int['shah'] ])

[-0.25362054 -1.0438123   0.2856574   0.6163969   1.9102874  -0.6714765
  0.62518835 -2.314383   -1.5104935   0.8064189  -1.1913656  -1.4861152
 -0.5942467   1.0724595   0.43549317  1.1082484   0.6819373  -1.2191142
 -0.7689338  -0.15788618 -0.8131546   0.16172361  0.91967905  0.33424133
  0.38527542  1.1702173   0.08115008  0.6969394   0.27155718 -0.40890798
  1.2671419   1.4491729   1.3631774   1.0906304   1.6149623   0.567616
  0.31402162 -0.9644282   1.2199166   0.75100034 -0.9217936  -0.5413823
  0.19473445 -0.16725743  0.19499421  0.11359031 -0.68131053  0.36611733
  3.3363757   0.46275818]


In [0]:
def euclidean_dist(vec1, vec2):
    return np.sqrt(np.sum((vec1-vec2)**2))

def find_closest(word_index, vectors):
    min_dist = 10000 # to act like positive infinity
    min_index = -1
    query_vector = vectors[word_index]
    for index, vector in enumerate(vectors):
        if euclidean_dist(vector, query_vector) < min_dist and not np.array_equal(vector, query_vector):
            min_dist = euclidean_dist(vector, query_vector)
            min_index = index
    return min_index

In [109]:
print(int2word[find_closest(word2int['khan'], vectors)])

born
